# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [159]:
#url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
#s = requests.get(url).content
#print(s)
#df=pd.read_csv(io.StringIO(s.decode('UTF8')))
### Если не работает загрузка on-line
df=pd.read_csv("Химический анализ родника в Нескучном саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [160]:
# Ваше решение
temp_df = df['Норматив'].str.rsplit(n=1, expand=True)
temp_AB = temp_df.iloc[:,1].str.split('-', expand=True)
df['Оператор'] = temp_df.iloc[:,0]
df['A'] = temp_AB.iloc[:,0].str.replace(',','.')
df['B'] = temp_AB.iloc[:,1].str.replace(',','.')
df

,Показатель,Единица измерений,Результат анализа,Норматив,Оператор,A,B
0,pH,единицы pH,8.4,в пределах 6-9,в пределах,6,9
1,Запах,баллы,1,не более 2-3,не более,2,3
2,Цветность,градусы,б/цвета,не более 30,не более,30,None
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,в пределах,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",не более,1.5,None
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,None
6,Нитраты (по NO3),мг/дм3,24,не более 45,не более,45,None
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5",не более,3.5,None
8,Хлориды (Cl),мг/дм3,200,не более 350,не более,350,None
9,Сульфаты (SO4),мг/дм3,189.5,не более 500,не более,500,None


In [161]:
def Decision(res, op, a, b):
    if op == 'не более':
        return (res <= max (a, b))
    elif op == 'в пределах':
        return (min (a, b) <= res) & (res <= max (a, b))
    else: return None

In [162]:
df['Результат анализа'] = pd.to_numeric(df['Результат анализа'], downcast='float', errors='coerce')
df['A'] = pd.to_numeric(df['A'], downcast='float')
df['B'] = pd.to_numeric(df['B'], downcast='float')
df['Оператор'] = df['Оператор'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Показатель         14 non-null     object 
 1   Единица измерений  14 non-null     object 
 2   Результат анализа  13 non-null     float32
 3   Норматив           14 non-null     object 
 4   Оператор           14 non-null     string 
 5   A                  14 non-null     float32
 6   B                  3 non-null      float32
dtypes: float32(3), object(3), string(1)
memory usage: 744.0+ bytes


In [164]:
df['Норма'] = df.apply ( lambda x: Decision(x['Результат анализа'], x['Оператор'], x['A'], x['B']), axis= 1 )
df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,Оператор,A,B,Норма
Показатель,,,,,,,
pH,единицы pH,8.400,в пределах 6-9,в пределах,6.0,9.0,True
Запах,баллы,1.000,не более 2-3,не более,2.0,3.0,True
Цветность,градусы,NaN,не более 30,не более,30.0,NaN,False
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7.0,10.0,True
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,"не более 1,5",не более,1.5,NaN,True
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",не более,3.3,NaN,True
Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45.0,NaN,True
Фосфаты (P),мг/дм3,0.360,"не более 3,5",не более,3.5,NaN,True
Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350.0,NaN,True


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [233]:
import random

# Ваше решение
# заполняем ящик фруктами
Box = ["апельсин"]*5 + ['яблоко']*4

# задаем число экспериментов
n = 1000000

In [234]:
# формируем выборку
A = pd.DataFrame(map(lambda a: random.sample(Box, k=3), range(n)))
A

,0,1,2
0,апельсин,апельсин,яблоко
1,апельсин,апельсин,апельсин
2,яблоко,апельсин,яблоко
3,апельсин,апельсин,яблоко
4,яблоко,яблоко,апельсин
...,...,...,...
999995,апельсин,яблоко,апельсин
999996,яблоко,яблоко,апельсин
999997,апельсин,яблоко,яблоко
999998,апельсин,яблоко,апельсин


In [235]:
# считаем статистику выпадений черных шаров
P_oranges = len(A[(A[0] == 'апельсин')&(A[1] == 'апельсин')&(A[2] == 'апельсин')])/A.shape[0]
P_oranges

0.119077

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [ ]:
# Ваше решение


In [236]:
Box = ["стандарт"]*7 + ['не стандарт']*3
n = 1000000
A = pd.DataFrame(map(lambda a: random.sample(Box, k=2), range(n)))
A

,0,1
0,стандарт,стандарт
1,стандарт,стандарт
2,стандарт,стандарт
3,стандарт,стандарт
4,стандарт,не стандарт
...,...,...
999995,стандарт,не стандарт
999996,стандарт,стандарт
999997,стандарт,не стандарт
999998,стандарт,не стандарт


In [237]:
P = len(A[(A[0] == 'не стандарт')&(A[1] == 'стандарт')])/A.shape[0]
P

0.233525